In [3]:
#Name: Md Bin Amin Hossan.
#Id: HOS20505803
#Artificial Intelligence - CMP020N206S


In [8]:
from collections import deque 
from queue import PriorityQueue
import heapq
from queue import heappush
from queue import heappop


# Define the map and distances between cities
distances = {
    'Manchester': {'Carlisle': 120, 'Liverpool': 40, 'York': 70, 'Newcastle': 140},
    'Holyhead': {'Liverpool': 90},
    'Liverpool': {'Manchester': 40, 'Holyhead': 90},
    'York': {'Manchester': 70, 'Newcastle': 80},
    'Carlisle': {'Glasgow': 100, 'Manchester': 120},
    'Newcastle': {'York': 80, 'Manchester': 140, 'Edinburgh': 110},
    'Glasgow': {'Carlisle': 100, 'Oban': 100, 'Edinburgh': 50, 'Inverness': 170, 'Aberdeen':140},
    'Edinburgh': {'Newcastle': 110, 'Glasgow': 50, 'Manchester': 220},
    'Oban': {'Glasgow': 100, 'Inverness': 110},
    'Aberdeen': {'Glasgow': 140, 'Inverness': 110},
    'Inverness': {'Inverness': 170, 'Aberdeen': 110, 'Oban': 110},
}

# Define the heuristic distances
heuristics = {
    'Manchester': 268,
    'Holyhead': 185,
    'Liverpool': 231,
    'York': 203,
    'Carlisle': 146,
    'Newcastle': 123,
    'Glasgow': 85,
    'Edinburgh': 55,
    'Oban': 23,
    'Aberdeen': 10,
    'Inverness': 0
}  

# Create a queue for BFs
def bfs(distances, heuristics, start, end):
        # Create a priority queue for A* search
    queue = [(heuristics[start], 0, start, [start])]
    
    # Set to keep track of visited cities
    visited = set()
    
    while queue:
        # Dequeue a city and its path
        _, cost, city, path = heappop(queue)
        
        # Check if the city has been visited before
        if city in visited:
            continue
        
        # Mark the city as visited
        visited.add(city)
        
        # Check if the current city is the end city
        if city == end:
            return path, cost
        
        # Enqueue all unvisited cities adjacent to the current city
        for neighbor, distance in distances[city].items():
            if neighbor not in visited:
                # Calculate the total cost of reaching the neighbor
                neighbor_cost = cost + distance
                
                # Calculate the heuristic cost from the neighbor to the end city
                neighbor_heuristic = heuristics[neighbor]
                
                # Enqueue the neighbor with its total cost and path
                heappush(queue, (neighbor_cost + neighbor_heuristic, neighbor_cost, neighbor, path + [neighbor]))
    
    # No path exists between the start and end cities
    return None, None

def dfs(distances, start, end, path=None, cost=0):
    if path is None:
        path = [start]

    if start == end:
        return path, cost

    for city in distances[start]:
        if city not in path:
            new_cost = cost + distances[start][city]
            new_path, new_cost = dfs(distances, city, end, path + [city], new_cost)
            if new_path:
                return new_path, new_cost

    return None, None

def astar(distances, heuristics, start, end):
    # Create a priority queue for A*
    queue = PriorityQueue()
    
    # Set to keep track of visited cities
    visited = set()
    
    # Initialize the start city
    queue.put((heuristics[start], start, [start]))
    
    while not queue.empty():
        # Dequeue a city and its path
        _, city, path = queue.get()
        
        # Check if the city has been visited before
        if city in visited:
            continue
        
        # Mark the city as visited
        visited.add(city)
        
        # Check if the current city is the end city
        if city == end:
            cost = sum(distances[path[i]][path[i+1]] for i in range(len(path)-1))

            return path, cost
        
        # Enqueue all unvisited cities adjacent to the current city
        for neighbor in distances[city]:
            if neighbor not in visited:
                # Calculate the distance and heuristic cost to the neighbor city
                cost = distances[city][neighbor]
                heuristic = heuristics[neighbor]
                total_cost = cost + heuristic
                
                # Add the neighbor city to the priority queue
                queue.put((total_cost, neighbor, path + [neighbor]))
    
    # No path exists between the start and end cities
    return None, None

def dijkstra(distances, start, end):
    queue = [(0, start, [])]
    visited = set()

    while queue:
        (cost, current, path) = heapq.heappop(queue)

        if current in visited:
            continue

        path = path + [current]

        if current == end:
            return (path, cost)

        visited.add(current)

        for neighbor in distances[current]:
            if neighbor not in visited:
                total_cost = cost + distances[current][neighbor]
                heapq.heappush(queue, (total_cost, neighbor, path))

    return None, None



# Get the start and end cities from the user
start = input("Enter start:")
end = input("Enter end point:")

# Validate the input
while start not in distances or end not in distances:
    print("Invalid input, please enter valid start and end points.")
    start = input("Enter start:")
    end = input("Enter end point:")
print("*****City-Map*****")
# Find the shortest path using BFS
path, cost = bfs(distances, heuristics, start, end)
if path is not None:
    print("BFS Path:", path)
    print("Total distance:", cost)
else:
    print("No path found.")

# Find the shortest path using DFS  
path, cost = dfs(distances, start, end)
if path is not None:
    print("DFS Path:", path)
    print("Total distance:", cost)
else:
    print("No path found.")

# Find the shortest path using A* 
path, cost = astar(distances, heuristics, start, end)

if path:
    print("A* Path:", path)
    print("Total distance:", cost)
else:
    print("No path exists between the start and end cities.")


# Find the shortest path using Dijk    
path, cost = dijkstra(distances, start, end)
if path is not None:
    print("Dijkstra Path:", path)
    print("Total distance:", cost,"Miles" )
else:
    print("No path found.")



Enter start:Manchester
Enter end point:Inverness
*****City-Map*****
BFS Path: ['Manchester', 'Carlisle', 'Glasgow', 'Inverness']
Total distance: 390
DFS Path: ['Manchester', 'Carlisle', 'Glasgow', 'Oban', 'Inverness']
Total distance: 430
A* Path: ['Manchester', 'Newcastle', 'Edinburgh', 'Glasgow', 'Oban', 'Inverness']
Total distance: 510
Dijkstra Path: ['Manchester', 'Carlisle', 'Glasgow', 'Inverness']
Total distance: 390 Miles


In [10]:
import folium
from geopy.geocoders import Nominatim
from geopy import distance

geolocator = Nominatim(user_agent="map_app")
start_location = geolocator.geocode(input("Enter starting location: "))
end_location = geolocator.geocode(input("Enter ending location: "))

start_point = (start_location.latitude, start_location.longitude)
end_point = (end_location.latitude, end_location.longitude)

# Calculate the distance between the starting and ending points
distance_miles = distance.distance(start_point, end_point).miles

# Calculate the time to travel between the starting and ending points (assuming a speed of 60 miles per hour)
time_hours = distance_miles / 60.0

map = folium.Map(location=[start_location.latitude, start_location.longitude], zoom_start=12)

folium.Marker([start_location.latitude, start_location.longitude], popup="Starting Point").add_to(map)
folium.Marker([end_location.latitude, end_location.longitude], popup="Ending Point").add_to(map)

# Add a straight line between the starting and ending points
line_points = [[start_location.latitude, start_location.longitude], [end_location.latitude, end_location.longitude]]
folium.PolyLine(locations=line_points, color='red', weight=5).add_to(map)

# Add text to the map displaying the distance and time between the starting and ending points
folium.map.Marker(
    [(start_location.latitude+end_location.latitude)/2, (start_location.longitude+end_location.longitude)/2],
    icon=folium.Icon(color='blue', icon='info-sign'),
    popup=f"Distance: {distance_miles:.2f} miles\nTime: {time_hours:.2f} hours"
).add_to(map)

display(map)


Enter starting location: London
Enter ending location: York
